In [1]:
# For Data
from Data.historical_data import Historical_data
from alpaca.data.timeframe import TimeFrame
from alpaca.data.historical import CryptoHistoricalDataClient, StockHistoricalDataClient

import pandas as pd
from Preprocessing.preprocessing import preprocessing_improved
from Models.LSTM.lightningLSTM import LightningLSTM
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from Preprocessing.dataclasses import StockPriceDataModule, StockDataset
import pytorch_lightning as pl

ModuleNotFoundError: No module named 'alpaca'

In [2]:
new_data = False

if new_data:

    stocks = [['AAPL'], ['MSFT'], ['GOOG'], ['TSLA'], ['NVDA']]

    for stock in stocks:

        timeframes = [TimeFrame.Day, TimeFrame.Hour, TimeFrame.Minute]
        time_strings = ['Day', 'Hour', 'Minute']

        for timeframe, time_string in zip(timeframes, time_strings):

            s_type = "StockBars"
            stock = stock
            timeframe = timeframe
            start = "2020-08-20 06:00:00"
            end = "2021-08-20 06:00:00"
            client = StockHistoricalDataClient("PKV7BABG3DGE0GYL137W", "af1eQghpVgbyX2lh2T9n2ColmVYXHCWndQbKR3Lr")
            save_csv = True
            time_string = time_string

            df = Historical_data(s_type = s_type, stock = stock, timeframe = timeframe, start = start, end = end, client = client, save_csv = save_csv , time_string = time_string)

else:

    df = pd.read_csv('Data\Stock\StockBars\MSFT_Minute')

In [3]:
data_train, data_test = preprocessing_improved(df = df[:int(len(df)*0.1)], lag = 1, sequence_length=100, dif_all=True, train_size=0.9 )
data_module = StockPriceDataModule(train_sequence = data_train,  test_sequence= data_test, batch_size=8, num_workers = 4)
data_module.setup()

for item in data_module.train_dataloader():
    print(item.keys())
    print(item['sequence'].shape)
    print(item['label'].shape)
    break


dict_keys(['sequence', 'label'])
torch.Size([8, 100, 8])
torch.Size([8])


In [4]:
model = LightningLSTM(input_size=8, hidden_size = 10, num_layers = 1)
trainer = pl.Trainer(max_epochs=5)
trainer.fit(model, data_module)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type    | Params
--------------------------------------
0 | model     | LSTM    | 811   
1 | criterion | MSELoss | 0     
--------------------------------------
811       Trainable params
0         Non-trainable params
811       Total params
0.003     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
